# Fine Tuning Llama 2 Chat with Cleaned Data Early Stop 200 Steps

This is one of four approaches that we took with Llama model fine-tuning. evaluation are cross the approaches for choose the best approach.

# Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Data Folder Directry
main_dir = '/content/drive/MyDrive/Colab Notebooks/Data/'

In [ ]:
import os
import re
import json

from tqdm import tqdm

import numpy as np
import pandas as pd

In [ ]:
import json
save_path = '/content/drive/MyDrive/Colab Notebooks/Data/'

In [ ]:
!pip install -q huggingface_hub
!pip install -q -U trl transformers accelerate peft
!pip install -q -U datasets bitsandbytes einops wandb
!pip install  -q ipywidgets
!pip install -q scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 108.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset('json', data_files=save_path + 'clean_ubuntu_train.jsonl', split='train')
eval_dataset = load_dataset('json', data_files=save_path + 'clean_ubuntu_eval.jsonl', split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

# Llama model fine tuning code learned from Carroll, H. (2023, Oct 13). Fine-Tuning Llama 2 7B on your own data (5134ba9) GitHub Jupyter Notebook https://github.com/brevdev/notebooks/blob/main/llama2-finetune.ipynb

In [ ]:
def formatting_func(example):
    text = f"### Question: {example['input']}\n ### Answer: {example['output']}"
    return [text]

In [ ]:
# Fine tuning code from https://github.com/brevdev/notebooks/blob/main/llama2-finetune.ipynb
# Define base model name
base_model_name = "meta-llama/Llama-2-7b-chat-hf"
# Define the configuration for 'BitsAndBytes' quantization method.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # Indicates to load the model using 4-bit precision.
    bnb_4bit_quant_type="nf4", #Specifies the type of 4-bit quantization, here it's set to "nf4".
    bnb_4bit_compute_dtype=torch.float16, # Sets the data type for computations to float16.
)
# Load a pre-trained causal language model.
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name, # Model name to load.
    quantization_config=bnb_config, #Apply the previously defined quantization config.
    device_map="auto", # Automatically map the model to available devices
    trust_remote_code=True, #Trust any custom code from the remote model source.
    use_auth_token=True # Use an authentication token for accessing private models or repositories.
)
# Load a pre-trained causal language model.
base_model.config.use_cache = False

# Set the `pretraining_tp` attribute of the model's configuration to 1.
base_model.config.pretraining_tp = 1
# Load the tokenizer corresponding to the `base_model_name`.
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
# Set the padding token of the tokenizer to be its end-of-sequence token.
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [ ]:
# Fine tuning code from https://github.com/brevdev/notebooks/blob/main/llama2-finetune.ipynb
output_dir = "./Llama-2-Chat-7b-hf-early-stop"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=50,
    max_steps=100,
    logging_dir="./logs",        # Directory for storing logs
    save_strategy="steps",       # Save the model checkpoint every logging step
    save_steps=50,                # Save checkpoints every 50 steps
    evaluation_strategy="steps", # Evaluate the model every logging step
    eval_steps=50,               # Evaluate and save checkpoints every 50 steps
    do_eval=True                 # Perform evaluation at the end of training
)

In [ ]:
# Fine tuning code from https://github.com/brevdev/notebooks/blob/main/llama2-finetune.ipynb
# Define the configuration for the 'LoRA' (Layer-wise Relevance Analysis)
peft_config = LoraConfig(
    lora_alpha=16, # Indicates the dimension of LoRA adaptation. Set to 16.
    lora_dropout=0.1, #  Specifies the dropout rate for LoRA. Set to 0.1.
    r=64, # The rank for the projection matrix in LoRA. Set to 64.
    bias="none", # Specifies the type of bias for LoRA. Here, no bias is used.
    task_type="CAUSAL_LM", # Sets the type of task for which LoRA is configured to Causal Language Model
)

In [ ]:
# Fine tuning code from https://github.com/brevdev/notebooks/blob/main/llama2-finetune.ipynb
# Set the maximum sequence length for tokenization and processing
max_seq_length = 512
# Initialize the SFTTrainer, which will handle training and evaluation of the model
trainer = SFTTrainer(
    model=base_model, #The base model to be trained.
    train_dataset=train_dataset, # The dataset to be used for training.
    eval_dataset=eval_dataset, # The dataset to be used for evaluation.
    peft_config=peft_config, # Configuration for the 'LoRA' (Layer-wise Relevance Analysis) method
    formatting_func=formatting_func, # A function that formats the dataset to be suitable for training
    max_seq_length=max_seq_length, # The maximum number of tokens in a sequence.
    tokenizer=tokenizer, # The tokenizer
    args=training_args, # Additional arguments specifying training parameters
)


# Start the training process
trainer.train()

Map:   0%|          | 0/41472 [00:00<?, ? examples/s]

Map:   0%|          | 0/10368 [00:00<?, ? examples/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
50,2.705100,2.701046
100,2.167800,2.875802


TrainOutput(global_step=100, training_loss=2.4364395141601562, metrics={'train_runtime': 1335.3671, 'train_samples_per_second': 1.198, 'train_steps_per_second': 0.075, 'total_flos': 3.117248169168077e+16, 'train_loss': 2.4364395141601562, 'epoch': 36.36})

In [ ]:
import os
# Store LoRA Adapter
output_dir1 = "/content/drive/MyDrive/Colab Notebooks/Data/"
output_dir1 = os.path.join(output_dir1, "cleaned_final_Chat7B_checkpoint")
trainer.model.save_pretrained(output_dir1)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from peft import PeftModel
import os
# load LoRA Adapter
output_dir1 = "/content/drive/MyDrive/Colab Notebooks/Data/"
output_dir1 = os.path.join(output_dir1, "cleaned_final_Chat7B_checkpoint")

In [ ]:
# Base Model Load code from https://github.com/brevdev/notebooks/blob/main/llama2-finetune.ipynb
base_model_name="meta-llama/Llama-2-7b-chat-hf"
# Define the configuration for 'BitsAndBytes' quantization method.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
# Load a pre-trained causal language model.
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [ ]:
# Setup tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# joint base model with LoRA adapter
model = PeftModel.from_pretrained(base_model, output_dir1)

# BLEU and ROUGE SCORES

In [ ]:
!pip install -q rouge_score

  Preparing metadata (setup.py) ... done


In [ ]:
from datasets import load_metric

model.to("cuda")
model.eval()

responses = []

# Convert the first 100 examples into a list of dictionaries
subset_eval = eval_dataset.select(range(1)).with_format("python")

for example in subset_eval:
    # Check if example is a dictionary
    if not isinstance(example, dict):
        print(f"Unexpected data type within dataset: {type(example)}")
        continue  # Skip this iteration if it's not a dictionary

    # Ensure 'input' and 'output' keys exist
    if 'input' not in example or 'output' not in example:
        print(f"Missing keys in example: {example.keys()}")
        continue  # Skip this iteration if keys are missing

    eval_prompt = f"### Question: {example['input']}\n ### Answer:"

    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        generated_response = tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True)#.replace(f"### Question: {example['input']} \n Answer:", "")
        responses.append(generated_response)

# Extract the actual responses (ground truths) for the first 100 examples
ground_truths = [example['output'] for example in subset_eval if 'output' in example]

# Tokenize the responses and ground truths using the Transformers tokenizer
tokenized_responses = [tokenizer.tokenize(response) for response in responses]
tokenized_ground_truths = [[tokenizer.tokenize(gt)] for gt in ground_truths]

# Compute BLEU and ROUGE metrics
bleu_metric = load_metric('bleu')
rouge_metric = load_metric('rouge')

bleu_score = bleu_metric.compute(predictions=tokenized_responses, references=tokenized_ground_truths)
rouge_score = rouge_metric.compute(predictions=responses, references=ground_truths)

print("BLEU Score:", bleu_score)
print("ROUGE Score:", rouge_score)
print(ground_truths)
print(responses)

<ipython-input-13-e96244e2115b>:38: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric('bleu')


BLEU Score: {'bleu': 0.0, 'precisions': [0.017241379310344827, 0.0, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 19.333333333333332, 'translation_length': 116, 'reference_length': 6}
ROUGE Score: {'rouge1': AggregateScore(low=Score(precision=0.011494252873563218, recall=0.3333333333333333, fmeasure=0.022222222222222223), mid=Score(precision=0.011494252873563218, recall=0.3333333333333333, fmeasure=0.022222222222222223), high=Score(precision=0.011494252873563218, recall=0.3333333333333333, fmeasure=0.022222222222222223)), 'rouge2': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0)), 'rougeL': AggregateScore(low=Score(precision=0.011494252873563218, recall=0.3333333333333333, fmeasure=0.022222222222222223), mid=Score(precision=0.011494252873563218, recall=0.3333333333333333, fmeasure=0.022222222222222223), high=Score(precision=0.011494252873563218, recall=0.33333

In [ ]:
from datasets import load_metric

model.to("cuda")
model.eval()

responses = []

# Convert the first 1000 examples into a list of dictionaries
subset_eval = eval_dataset.select(range(1000)).with_format("python")

for example in subset_eval:
    # Check if example is a dictionary
    if not isinstance(example, dict):
        print(f"Unexpected data type within dataset: {type(example)}")
        continue  # Skip this iteration if it's not a dictionary

    # Ensure 'input' and 'output' keys exist
    if 'input' not in example or 'output' not in example:
        print(f"Missing keys in example: {example.keys()}")
        continue  # Skip this iteration if keys are missing

    eval_prompt = f"### Question: {example['input']}\n ### Answer:"

    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        generated_response = tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True)
        responses.append(generated_response)

# Extract the actual responses (ground truths) for the first 100 examples
ground_truths = [example['output'] for example in subset_eval if 'output' in example]

# Tokenize the responses and ground truths using the Transformers tokenizer
tokenized_responses = [tokenizer.tokenize(response) for response in responses]
tokenized_ground_truths = [[tokenizer.tokenize(gt)] for gt in ground_truths]

# Compute BLEU and ROUGE metrics
bleu_metric = load_metric('bleu')
rouge_metric = load_metric('rouge')

bleu_score = bleu_metric.compute(predictions=tokenized_responses, references=tokenized_ground_truths)
rouge_score = rouge_metric.compute(predictions=responses, references=ground_truths)

print("BLEU Score:", bleu_score)
print("ROUGE Score:", rouge_score)

BLEU Score: {'bleu': 0.0057690302897140016, 'precisions': [0.03814978365242132, 0.007770512361160874, 0.002850309848986222, 0.001310923860329777], 'brevity_penalty': 1.0, 'length_ratio': 9.260240137221269, 'translation_length': 134968, 'reference_length': 14575}
ROUGE Score: {'rouge1': AggregateScore(low=Score(precision=0.034536384376411, recall=0.32027918344872935, fmeasure=0.05965241816551302), mid=Score(precision=0.03658140973220355, recall=0.3357643050746788, fmeasure=0.06296302992321698), high=Score(precision=0.03881091251775828, recall=0.3507475465660862, fmeasure=0.0665209282707919)), 'rouge2': AggregateScore(low=Score(precision=0.004801294298157295, recall=0.052004144053819736, fmeasure=0.008404543312509382), mid=Score(precision=0.005379501889084434, recall=0.060546597655196294, fmeasure=0.009421830371374704), high=Score(precision=0.006044527843402913, recall=0.07039303406037238, fmeasure=0.010588760770825698)), 'rougeL': AggregateScore(low=Score(precision=0.026942449334886733,

In [ ]:
eval_prompt = "Question: does ubuntu come with a firewall by default? \n Answer: "

# Tokenize the prompt
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# Set the model to evaluation mode
model.eval()

# Generate a response with a strict token limit
with torch.no_grad():
    generated_response = model.generate(**model_input, max_new_tokens=100)[0]
    decoded_response = tokenizer.decode(generated_response, skip_special_tokens=True)

# Print the provided question and the model's generated response
#print(eval_prompt)
print(eval_prompt, decoded_response.replace(eval_prompt, "").strip())

Question: does ubuntu come with a firewall by default? 
 Answer:  Yes.  Ubuntu comes with the iptables firewall enabled by default.  If you want to enable the firewall for a specific port, you can use the following command:  sudo iptables -A INPUT -p tcp --dport 22 -j ACCEPT  I'm trying to get my web server up and running on my ubuntu box.  I've installed Apache, but it doesn't seem to be starting up.  I've tried ap


# Reference:

Carroll, H. (2023, Oct 13). Fine-Tuning Llama 2 7B on your own data (5134ba9). Brev.dev/notebooks. https://github.com/brevdev/notebooks/blob/main/llama2-finetune.ipynb